In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import plotly.graph_objects as go

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = 'aacuser'
password = 'password'

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='hidden-div', style={'display': 'none'}),
            html.A([
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                        height=100, width=100
                        )
                ],
                    href='https://www.snhu.edu', target='blank'),
                html.Center(html.B(html.H1(children=[
                    'July Wellman',
                    html.Br(),
                    'SNHU - CS-340',
                    html.Br(),
                    'Grazioso Salvare Dashboard'
                ])))
            ]),
    
    html.Hr(),
    html.Div(
        
#code for the interactive filtering options
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset All', 'value': 'RESET'},
                {'label': 'Water Rescue', 'value': 'WaterRescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'WildernessRescue'},
                {'label': 'Disaster or Individual Tracking Rescue', 'value': 'DisasterRescue'}
            ]
        )
    ),
    
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable = False, 
        filter_action='native',
        sort_action = 'native',
        sort_mode='multi',
        column_selectable = False,
        row_selectable = 'single',
        row_deletable = False, 
        selected_columns = [],
        selected_rows = [],
        page_action = 'native',
        page_current = 0,
        page_size = 10
    ),
    
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
                html.Div([
                    #Dropdown for Pie Chart
                    dcc.Dropdown(
                        id='pie_dropdown',
                        className='col s12 m6',
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Age', 'value': 'age_upon_outcome'},
                            {'label': 'Outcome_type', 'value': 'outcome_type'}
                        ],
                        value='breed',
                        multi=False,
                        clearable=False
                        )
                    ],
                style={"width":"10%"}),
                #Pie Chart
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    style={"width":"45%"}),
                #Geolocation Map
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                style={"width":"45%"})
            ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([
    Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    Output('datatable-id', 'selected_rows')],
    [Input('filter-type', 'value')])

def update_dashboard(filter_type):
##code to filter interactive data table with MongoDB queries
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))
    elif filter_type == 'WaterRescue':
        df = pd.DataFrame(list(shelter.read({
            "animal_type": 'Dog',
            'breed': {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0,"$lte": 156.0}
        })))
        
    elif filter_type == "WildernessRescue":
        df = pd.DataFrame(list(shelter.read({
            "animal_type": 'Dog',
            'breed': {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0,"$lte": 156.0}
        })))
        
    elif filter_type == "DisasterRescue":
        df = pd.DataFrame(list(shelter.read({
            "animal_type": 'Dog',
            'breed': {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever","Bloodhound","Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0,"$lte": 300.0}
        })))
        
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    
    data = df.to_dict('records')
    columns = [{"name": i,"id": i,"deletable": False,"selectable": True} for i in df.columns]
    selected_rows=[0]
    return (data, columns, selected_rows)

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')])

def update_styles(selectedRows):
    return[{
        'if': {'row_index': i},
        'background_color': '#D2F3FF'
    }for i in selectedRows]

    
#Pie Chart callback
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
    Input('pie_dropdown', 'value')])
def update_graphs(viewData, dropdownValue):
    dff=pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure=px.pie(
                data_frame=dff,
                names=dropdownValue))]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "selected_rows")])
def update_map(viewData, selectedRows):  
    if not selectedRows:
        selected_row = 0
    else:
        selected_row = selectedRows[0]
    
    dff = pd.DataFrame.from_dict(viewData)
    # get position from selected row
    posX = dff.iloc[selected_row, 13]
    posY = dff.iloc[selected_row, 14]
    
    name = dff.iloc[selected_row, 9]
    if name =="":
        name = "No Name"
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[posX,posY], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[posX,posY], children=[
                dl.Tooltip(dff.iloc[selected_row,4]),
                dl.Popup([
                    html.P("Animal Name"),
                    html.H1(name)
                ])
            ])
        ])
    ]



app.run_server(debug=True)
